In [8]:
'''
对T+CLOUD上的收发存汇总表进行二次开发，与公司以前的格式一致
此版本加零结存，将抄本中的小学生按原顺序排列
'''
import os
import datetime
import easygui
import numpy as np
import pandas as pd
import openpyxl
from openpyxl.styles import Font, Border, Side, Fill, Alignment
import formatPainter

# store_dic = {'001库':"df.num == '001'",'002电商库':"df.num == '002'",'总库':"(df.num == '001') | (df.num == '002')"}
lst = ['仓库编码',
       '仓库',
       '存货分类 (1级)',
       '存货分类 (2级)',
       '存货分类 (3级)',
       '存货分类 (4级)',
       '存货分类 (5级)',
       '存货编码',
       '存货',
       '存货代码',
       '数量(主单位)',
       '平均单价',
       '金额',
       '数量(辅单位)',
       '计量单位组合',
       '数量(主单位).1',
       '平均单价.1',
       '金额.1',
       '数量(辅单位).1',
       '计量单位组合.1',
       '数量(主单位).2',
       '平均单价.2',
       '金额.2',
       '数量(辅单位).2',
       '计量单位组合.2',
       '数量(主单位).3',
       '平均单价.3',
       '金额.3',
       '辅数量',
       '计量单位组合.3']
lst2 = [
    'store',
    'num',
    'class01',
    'class02',
    'class03',
    'class04',
    'class05',
    'code',
    'stock',
    'content',
    'begin_ben',
    '平均单价',
    '金额',
    'begin_jian',
    '计量单位组合',
    'ruku_ben',
    '平均单价.1',
    '金额.1',
    'ruku_jian',
    '计量单位组合.1',
    'chuku_ben',
    '平均单价.2',
    '金额.2',
    'chuku_jian',
    '计量单位组合.2',
    'end_ben',
    '平均单价.3',
    '金额.3',
    'end_jian',
    '计量单位组合.3']

lst3 = [
    'class02',
    'class05',
    'code',
    'stock',
    'content',
    'begin_ben',
    'begin_jian',
    'ruku_ben',
    'ruku_jian',
    'chuku_ben',
    'chuku_jian',
    'end_ben',
    'end_jian',
]

lst_yeterday_ruku = ['class02',
 'class05',
 'code',
 'stock',
 'begin_ben',
 'begin_jian',
 'ruku_ben',
 'ruku_jian',
 ]
lst_yeterday_ruku1 = ['class02',
 'class05',
 'code',
 'stock',
 'yesterday_ben',
 'yesterday_jian',
 'yesterday_ruku_ben',
 'yesterday_ruku_jian',
 ]

lst01_yesterday = [
    'class02',
    'class05',
    'code',
    'stock',
    'begin_ben',
    'begin_jian',
    'ruku_ben',
    'ruku_jian',
    'chuku_ben',
    'chuku_jian',
]
lst02_yesterday = [
    'class02',
    'class05',
    'code',
    'stock',
    'yesterday_ben',
    'yesterday_jian',
    'yesterday_ruku',
    'yesterday_chuku',

]
lst_merge0 = ['class02',
 'class05',
 'code',
 'stock',
 'content',
 'begin_ben',
 'begin_jian',
 'ruku_ben',
 'ruku_jian',
 'chuku_ben',
 'chuku_jian',
 'end_ben',
 'end_jian',
 'yesterday_ben',
 'yesterday_jian',
 'yesterday_ruku_ben',
 'yesterday_ruku_jian',
 'yesterday_chuku_ben',
 'yesterday_chuku_jian']

lst_merge = ['class02', 'class05', 'code', 'stock',
             'content',
             'begin_jian', 'yesterday_jian',
             'yesterday_ruku_jian', 'ruku_jian',
             'yesterday_chuku_jian', 'chuku_jian',
             'end_jian']

lst_result0 = ['class05', 'code', 'stock',
               'content',
               'begin_jian', 'yesterday_jian',
               'yesterday_ruku_jian', 'ruku_jian',
               'yesterday_chuku_jian', 'chuku_jian',
               'end_jian']

lst_result = ['类别', '货号', '品名',
              '含量',
              '月初', '上日',
              '本日入库', '入库累计',
              '本日出库', '出库累计',
              '结余']


def chuli(fname, store_num):
    df = pd.read_excel(fname, skiprows=7)
    df = df.iloc[:, 1:]
    df.columns = lst
    df.columns = lst2
    if store_num == '001库':
        df = df.loc[df.store == '001']
    elif store_num == '002电商库':
        df = df.loc[df.store == '002']
    else:
        df = df.loc[(df.store == '001') | (df.store == '002')]

    df['content'] = df['end_ben'] / df['end_jian']
    df = df[df['store'] != '制表人:']
    df = df[df['store'] != '合计：']
    df = df[df['store'].notnull()]
    df = df.iloc[:, 2:]
    df = df[lst3]
    return df


# first,choice仓库编号
msg = '请点选仓库'
nums = ['001库', '002电商库', '总库']
num = easygui.buttonbox(msg, choices=nums)
msg = '请点选产成品"当日"工作表'
fname = easygui.fileopenbox(msg, title='AAA')
path, filename = os.path.split(fname)
os.chdir(path)
df1 = chuli(fname, num)
msg = '请点选产成品"累计"工作表'
fname2 = easygui.fileopenbox(msg, title='BBB')
df2 = chuli(fname2, num)
#当日汇总表仅取入库数
df1 = df1[lst_yeterday_ruku]
df1.columns = lst_yeterday_ruku1

today_date = datetime.date.today()
yesterday_date = (today_date + datetime.timedelta(days=-1)).strftime("%Y-%m-%d")
msg = '报表日期是{}?'.format(yesterday_date)
choice = easygui.ccbox(msg, title='请选择"是"或"否"', choices=('是', '否'))
if choice:
    riqi = yesterday_date
else:
    msg = '请输入报表日期'
    riqi = easygui.enterbox(msg, title=" 昨天日期")
fname_canchengpin = '产成品日报表{}.xlsx'.format(riqi)
wb = openpyxl.Workbook()
wb.save(fname_canchengpin)
#连接累计汇总表和当日汇总表
df2_df1 = pd.merge(df2, df1, how='left', on=['class02', 'class05', 'code', 'stock'], sort=False)
df2_df1
  
      





,class02,class05,code,stock,content,begin_ben,begin_jian,ruku_ben,ruku_jian,chuku_ben,chuku_jian,end_ben,end_jian,yesterday_ben,yesterday_jian,yesterday_ruku_ben,yesterday_ruku_jian
0,抄本,材料纸,DB501,单行材料纸,500.0,82000.0,164.0,NaN,NaN,7000.0,14.0,75000.0,150.0,75000.0,150.0,NaN,NaN
1,抄本,材料纸,FB503,方格材料纸,500.0,58000.0,116.0,NaN,NaN,2000.0,4.0,56000.0,112.0,56000.0,112.0,NaN,NaN
2,抄本,材料纸,J1601,简装双行材料纸,270.0,21870.0,81.0,NaN,NaN,540.0,2.0,21330.0,79.0,21330.0,79.0,NaN,NaN
3,抄本,材料纸,J1602,简装单行材料纸,270.0,31050.0,115.0,NaN,NaN,270.0,1.0,30780.0,114.0,30780.0,114.0,NaN,NaN
4,抄本,材料纸,J1603,简装方格材料纸,270.0,14580.0,54.0,NaN,NaN,810.0,3.0,13770.0,51.0,13770.0,51.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644,账本,装订配件,L16KPP,16KPP账夹（立信孔）,260.0,5200.0,20.0,NaN,NaN,NaN,NaN,5200.0,20.0,5200.0,20.0,NaN,NaN
1645,账本,装订配件,存料卡,48K存料卡,5400.0,280800.0,52.0,NaN,NaN,10800.0,2.0,270000.0,50.0,270000.0,50.0,NaN,NaN
1646,账本,装订配件,固定资产明细卡,30K固定资产明细卡,3000.0,48000.0,16.0,NaN,NaN,NaN,NaN,48000.0,16.0,48000.0,16.0,NaN,NaN
1647,账本,装订配件,商品卡,48K商品卡,5400.0,113400.0,21.0,NaN,NaN,NaN,NaN,113400.0,21.0,113400.0,21.0,NaN,NaN


In [9]:
msg = '请点选工厂昨天的销售订单'
fname_dingdan = easygui.fileopenbox(msg, title='工厂昨天的销售订单工作表')
df_dingdan = pd.read_excel(fname_dingdan)
df_dingdan= df_dingdan[df_dingdan['单据执行状态'] != '合计']
df_dingdan = df_dingdan[['存货编码','存货名称','存货分类','数量(本）','数量2（件）']]
df_dingdan.columns = ['code','stock','class05','yesterday_chuku_ben','yesterday_chuku_jian']
df_dingdan


,code,stock,class05,yesterday_chuku_ben,yesterday_chuku_jian
0,N0781（方格）,16K60型牛卡缝线语文本,订制产品,4000,20.00
1,N0783,16K60型牛卡缝线英语本,订制产品,3000,15.00
2,N0785,16K60型牛卡缝线练习本,订制产品,6000,30.00
3,N0784（300格）,16K60型牛卡缝线作文本,订制产品,800,4.00
4,N0786,16K60型牛卡缝线读书笔记,订制产品,2000,10.00
...,...,...,...,...,...
617,EXBB5100NB,B5-100型牛卡线环空白本（上翻）,牛卡线环本,960,10.00
618,EXWGB5100NB,B5-100型牛卡线环网格本（上翻）,牛卡线环本,192,2.00
619,EFHP3260N,32K60型牛卡缝线汉语拼音,32K牛卡缝线本,11200,35.00
620,A5-92页书芯t,A5-92页书芯t,领创未来,10584,105.84


In [10]:
#汇总销售订单
gp_dingdan = df_dingdan.groupby(['class05','code','stock'])
yesterday_chuku_ben	 = gp_dingdan['yesterday_chuku_ben'].sum()
yesterday_chuku_jian = gp_dingdan['yesterday_chuku_jian'].sum()
df_dingdan_total = pd.DataFrame({'yesterday_chuku_ben':yesterday_chuku_ben,'yesterday_chuku_jian':yesterday_chuku_jian}).reset_index()

In [11]:
#再连接销售订单
merge = pd.merge(df2_df1,df_dingdan_total,how = 'outer',on =['class05', 'code', 'stock'],\
                  sort=False)
merge.class02 = merge.class02.fillna(method = 'ffill')
merge = merge.fillna(0)
merge


C:\Users\redda\AppData\Local\Temp\ipykernel_29048\3489744683.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merge.class02 = merge.class02.fillna(method = 'ffill')


,class02,class05,code,stock,content,begin_ben,begin_jian,ruku_ben,ruku_jian,chuku_ben,chuku_jian,end_ben,end_jian,yesterday_ben,yesterday_jian,yesterday_ruku_ben,yesterday_ruku_jian,yesterday_chuku_ben,yesterday_chuku_jian
0,电商,16K牛卡缝线本,EFB1660NB,16K60型牛卡缝线空白本,200.0,7400.0,37.0,0.0,0.0,3000.0,15.0,4400.0,22.0,4400.0,22.0,0.0,0.0,0.0,0.0
1,电商,16K牛卡缝线本,EFHP1660N,16K60型牛卡缝线汉语拼音,200.0,32000.0,160.0,0.0,0.0,19000.0,95.0,13000.0,65.0,19000.0,95.0,0.0,0.0,8000.0,40.0
2,电商,16K牛卡缝线本,EFHP1660N(10本）,16K60型牛卡缝线汉语拼音（10本装）,200.0,28400.0,142.0,0.0,0.0,23000.0,115.0,5400.0,27.0,20400.0,102.0,0.0,0.0,15000.0,75.0
3,电商,16K牛卡缝线本,EFL1660NB-定制,16K60型牛卡缝线横线本,200.0,101800.0,509.0,26400.0,132.0,46000.0,230.0,82200.0,411.0,82200.0,411.0,0.0,0.0,0.0,0.0
4,电商,16K牛卡缝线本,EFPY1660N,16K60型牛卡缝线拼音本,200.0,23200.0,116.0,0.0,0.0,2000.0,10.0,21200.0,106.0,21200.0,106.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,电商,齐达创美定制,EFSX1660K,16K60型卡面五彩马卡龙缝线数学本,200.0,8400.0,42.0,0.0,0.0,4000.0,20.0,4400.0,22.0,4400.0,22.0,0.0,0.0,200.0,1.0
1650,电商,齐达创美定制,EFY1660K,16K60型卡面五彩马卡龙缝线英语本,0.0,2200.0,11.0,0.0,0.0,2200.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1651,电商,齐达创美定制,EFZW1660K,16K60型卡面五彩马卡龙缝线作文本,0.0,3800.0,19.0,0.0,0.0,3800.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1652,电商,齐达创美定制,ETHA430A,A4胶装图画本,140.0,12180.0,87.0,0.0,0.0,0.0,0.0,12180.0,87.0,12180.0,87.0,0.0,0.0,0.0,0.0


In [ ]:
merge

In [12]:
merge.content = (merge.begin_ben +  merge.ruku_ben)/(merge.begin_jian+  merge.ruku_jian)
merge.chuku_jian = merge.chuku_jian + merge.yesterday_chuku_jian
merge.end_jian = merge.end_jian - merge.yesterday_chuku_jian
merge = merge[lst_merge]




In [13]:
gp = merge.groupby('class02', sort=False)
data_heji = []
def xiaojiClass05(gp2, key, d):
    d = d[lst_result0]
    d.loc[key] = d.sum()
    d.iloc[-1, :4] = [key + '小计', '', '', '']
    return d


for name, gp002 in gp:
    table = pd.pivot_table(gp002, index=['class05', 'code'], aggfunc=sum, sort=False)
    table = table.reset_index()
    gp2 = table.groupby('class05', sort=False)

    heji = pd.DataFrame(gp002).sum()
    data = []
    for key, gp005 in gp2:
        df05 = xiaojiClass05(gp2, key, gp005)
        data.append(df05)
    df005 = pd.concat(data)
    df005 = df005[lst_result0]

    df005.loc[name] = heji
    df005.iloc[-1, :4] = [name + '合计', '', '', '']
    heji_row = df005.iloc[-1]
    data_heji.append(heji_row)

    with pd.ExcelWriter(fname_canchengpin, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        df005.columns = lst_result
        df005.to_excel(writer, sheet_name=name, index=False)
df_heji = pd.DataFrame(data_heji)
df_heji.columns = lst_result
df_heji.loc['合计'] = df_heji.sum(0)


C:\Users\redda\AppData\Local\Temp\ipykernel_29048\2169210939.py:11: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  table = pd.pivot_table(gp002, index=['class05', 'code'], aggfunc=sum, sort=False)
C:\Users\redda\AppData\Local\Temp\ipykernel_29048\2169210939.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.loc[key] = d.sum()
C:\Users\redda\AppData\Local\Temp\ipykernel_29048\2169210939.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  d.il

In [14]:
def res_path(relative_path):
    """获取资源绝对路径"""
    try:
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")
    return os.path.join(base_path, relative_path)
def printseting(fname1, riqi):
    # 设置字体、边框、对齐等常量
    font = Font(name='宋体', size=10)
    font_xiaoji = Font(name='宋体', bold=True, size=10)
    font_firstrow = Font(name='宋体', bold=True, size=20)
    thin_danbian = Side(border_style='thin')
    double_danbian = Side(border_style='double')
    thin_bian = Border(
        left=thin_danbian,
        right=thin_danbian,
        top=thin_danbian,
        bottom=thin_danbian)
    double_bian = Border(
        left=thin_danbian,
        right=thin_danbian,
        top=thin_danbian,
        bottom=double_danbian)
    wb = openpyxl.load_workbook(fname1)
    for ws in wb.worksheets:
        if ws.title == 'Sheet':
            del wb[ws.title]
        else:
            # 插入第一行
            ws.insert_rows(1)
            ws['A1'].value = '双佳' + ws.title + ' ' + riqi
            ws['A1'].font = font_firstrow
            ws['A1'].alignment = Alignment(horizontal='center', vertical='center')
            max_row = ws.max_row
            max_column = ws.max_column
            if ws.title != '合计':
                # 单元格宽度
                ws.column_dimensions['A'].width = 8.56
                ws.column_dimensions['B'].width = 17.33
                ws.column_dimensions['C'].width = 13.56
                ws.column_dimensions['D'].width = 3.67
                for j in 'EFGHIJK':
                    ws.column_dimensions['{}'.format(j)].width = 7.44
                    ws.merge_cells('A1:K1')
                for row in range(2, max_row + 1):  # 从第二行开始，设置单元格格式
                    if row == 2:
                        for col in range(1, max_column + 1):
                            ws.cell(row, col).alignment = Alignment(horizontal='center', vertical='center')
                    else:
                        for col in range(1, max_column + 1):
                            if col <= 3:
                                ws.cell(row, col).alignment = Alignment(horizontal='left', vertical='center')
                            else:
                                ws.cell(row, col).alignment = Alignment(horizontal='center', vertical='center')
                ws.freeze_panes = ws['E3']
            else:
                # 单元格宽度
                ws.column_dimensions['A'].width = 16
                ws.column_dimensions['B'].width = 12
                ws.column_dimensions['C'].width = 14
                ws.column_dimensions['D'].width = 12
                ws.column_dimensions['E'].width = 12
                ws.column_dimensions['F'].width = 12
                ws.column_dimensions['G'].width = 12
                ws.column_dimensions['H'].width = 12
                ws.merge_cells('A1:H1')
                # ws.freeze_panes = ws['E3']
                for row in range(2, max_row + 1):  # 从第二行开始，设置单元格格式
                    if row == 2:
                        for col in range(1, max_column + 1):
                            ws.cell(row, col).alignment = Alignment(horizontal='center', vertical='center')
                    else:
                        for col in range(1, max_column + 1):
                            if col <= 1:
                                ws.cell(row, col).alignment = Alignment(horizontal='left', vertical='center')
                            else:
                                ws.cell(row, col).alignment = Alignment(horizontal='right', vertical='center')
            for cell in ws['A']:
                row = cell.row
                if row == 1:
                    continue
                else:
                    if '小计' in cell.value:
                        for col in range(1, max_column + 1):
                            ws.cell(row, col).font = font_xiaoji
                            ws.cell(row, col).border = double_bian
                            ws.merge_cells('A{}:C{}'.format(row, row))
                    else:
                        for col in range(1, max_column + 1):
                            ws.cell(row, col).font = font
                            ws.cell(row, col).border = thin_bian
            # ws.freeze_panes = ws['E3']
            ws.print_title_rows = '1:2'  # the first row
            # 页脚设置
            ws.oddFooter.center.text = " &[Page] / &N"  # 1/n
            ws.oddFooter.center.size = 12  # 页脚中字体大小
            ws.oddFooter.center.font = "Tahoma"  # 页脚中字体
            ws.oddFooter.center.color = "000000"  # 页脚中字体颜色
            ws.oddFooter.right.text = "张文伟 &[Date]"  # 页脚右 文字
            ws.oddFooter.right.size = 12  # 页脚右 字体大小
            ws.oddFooter.right.font = "书体坊米芾体"  # 页脚右 字体
            ws.oddFooter.right.color = "000000"  # 页脚右 字体颜色
            ws.page_margins = openpyxl.worksheet.page.PageMargins(top=0.48, header=0.5, left=0.21, right=0.24,
                                                                  footer=0.5,
                                                                  bottom=1)
            for i in ws.iter_rows():  # 将零值替换为空
                for j in i:
                    if j.value == 0:
                        j.value = ''
                    else:
                        continue
    wb.save(fname1)




# 零结存
merge1 = merge[lst_merge]
merge1 = merge1.fillna(0)
df_zero = merge1[merge1.end_jian == 0]
df_zero = df_zero.iloc[:, 1:]
df_zero.columns = lst_result
df_zero.loc['合计'] = df_zero.sum(0)
df_zero.iloc[-1, :4] = ['零结存合计', '', '', '']
with pd.ExcelWriter(fname_canchengpin, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    df_zero.to_excel(writer, sheet_name='零结存', index=False)

with pd.ExcelWriter(fname_canchengpin, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    df_heji = df_heji[
        ['月初', '上日',
         '本日入库', '入库累计',
         '本日出库', '出库累计',
         '结余']]
    df_heji.to_excel(writer, sheet_name='合计', index_label='类别')

printseting(fname_canchengpin, riqi)
fname0 = res_path('img\leiji.xlsx')
wb0 = openpyxl.load_workbook(res_path('img\leiji.xlsx'))
ws0 = wb0.active
area0 = ws0['A1:H10']
wb = openpyxl.load_workbook(fname_canchengpin)
ws = wb['合计']
cell_start1 = ws['A1']
formatPainter.stylesFormat(ws, area0, cell_start1)
wb0.close()
wb.save(fname_canchengpin)
os.startfile(fname_canchengpin)


C:\Users\redda\AppData\Local\Temp\ipykernel_29048\4071728336.py:125: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_zero.iloc[-1, :4] = ['零结存合计', '', '', '']
